In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi
from scipy.signal import find_peaks

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.PlaceCalc import PlaceCalc

In [2]:
data = {}
place_calc = PlaceCalc(
    num_shuffles=200, threshold=0.99, dimension=1, bins_2d=None
    )
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        print(fil_string)
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        spatial_info, significance_1d, _ = place_calc.calc_place_cells(exp_data)
        data[fil_string]['spatial_info'] = spatial_info
        data[fil_string]['significance'] = significance_1d
        data[fil_string]['tuning'] = np.zeros((exp_data.num_neurs, 16)).astype(bool)
        data[fil_string]['mean_fr'] = np.zeros((exp_data.num_neurs, 16))
        for neur in range(exp_data.num_neurs):
            if not significance_1d[neur]: continue
            frs = []
            for wedge in (np.arange(16) + 1):
                wedge_frs = exp_data.fr[neur, exp_data.wedges == wedge]
                frs.append(wedge_frs)
            mean_frs = [np.nanmean(f) for f in frs]
            neur_tuning = np.argwhere(
                mean_frs > np.mean(mean_frs) + np.std(mean_frs)
                ).flatten()
            data[fil_string]['mean_fr'][neur, :] = mean_frs
            if not significance_1d[neur]: continue
            data[fil_string]['tuning'][neur, neur_tuning] = True
with open(pickle_dir / 'place_cells_1d.p', 'wb') as f: 
    pickle.dump(data, f)

/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184331_09102019.mat


/home/chingf/Code/cache-arena-analysis/analysis/tuning_measures/mutual_info.py:20: RuntimeWarning: divide by zero encountered in log2
  log_term = np.log2(condn_mean_fr/mean_fr)


/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned185033_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184713_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184430_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned144233_09112019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184526_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/ForChing/ExtractedWithXY_Cleaned184946_09102019.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200930/2020-09-30-11-00-47_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithOldBehavVariables154524_11062020.mat


/home/chingf/Code/cache-arena-analysis/analysis/tuning_measures/mutual_info.py:20: RuntimeWarning: invalid value encountered in true_divide
  log_term = np.log2(condn_mean_fr/mean_fr)
/home/chingf/Code/cache-arena-analysis/analysis/PlaceCalc.py:34: RuntimeWarning: invalid value encountered in true_divide
  spatial_info /= shuffled_spatial_info


/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20201009/2020-10-09-11-40-59_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithOldBehavVariables150539_11062020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20201012/2020-10-12-11-43-36_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithOldBehavVariables160403_11062020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200918/2020-09-18-11-50-00_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithOldBehavVariables160820_11062020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200908/2020-09-08-12-02-54_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithOldBehavVariables162633_11062020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20200910/2020-09-10-12-00-48_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWithOldBehavVariables162716_11062020.mat
/home/chingf/engram/Emily/NeuralData/Gcamp/LMN73/20201007/2020-10-07-11-48-53_video_SplitIntoBatches_29_250_10_200/Results/ExtractedWit